# Neural Network

##### Importing necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import PIL.Image
from PIL import ImageDraw, ImageOps
import os
import pandas as pd
from tkinter import *
from tkinter import filedialog
import numpy as np

print(torch.cuda.get_device_name(0)) # Name of my GPU Device
print(torch.cuda.is_available()) # Checking if CUDA is available


NVIDIA GeForce RTX 3060 Laptop GPU
True


##### Adding data points and labelling them

In [3]:
class MNISTDataset(Dataset):

    # Making a constructor to initialize the attributes

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        for label in range(10):  # Folders are named from '0' to '9'
            folder = os.path.join(root_dir, str(label))
            for filename in os.listdir(folder):
                filepath = os.path.join(folder, filename)
                self.data.append((filepath, label))


    # allowing len() to be used to an instance of the class

    def __len__(self):
        return len(self.data)

    # 

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = PIL.Image.open(img_path).convert('L')
        if self.transform:
            image = self.transform(image)
        return image, label

##### Defining the Neural Network

In [4]:
class Net(nn.Module):

    # Making a constructor to initialize the linear transformations

    def __init__(self):
        super(Net, self).__init__()
        self.IL_to_HL1 = nn.Linear(28 * 28, 128)
        self.HL1_to_HL2 = nn.Linear(128, 64)
        self.HL2_to_OL = nn.Linear(64, 10)

    # Defining how data flows through the layers

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.IL_to_HL1(x))
        x = F.relu(self.HL1_to_HL2(x))
        x = self.HL2_to_OL(x)
        return x

##### Resizing the image and creating datasets and dataloaders

In [5]:
train_transform = transforms.Compose([transforms.Resize((28, 28)),
                                      transforms.ToTensor()])

test_transform = transforms.Compose([transforms.Resize((28, 28)),
                                     transforms.ToTensor()])

train_dataset = MNISTDataset(root_dir=r'D:\Ashish\Math\Data Science\DataSet\mnist_png\training',
                             transform=train_transform)
test_dataset = MNISTDataset(root_dir=r'D:\Ashish\Math\Data Science\DataSet\mnist_png\testing',
                            transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

##### Training and Testing the model

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        target_one_hot = F.one_hot(target, num_classes=10).float()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target_one_hot)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

# Testing loop
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            target_one_hot = F.one_hot(target, num_classes=10).float()
            output = model(data)
            test_loss += criterion(output, target_one_hot).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)

# Save the trained model
torch.save(model.state_dict(), 'mnist_net_mse.pth')

Train Epoch: 1 [0/60000] Loss: 0.106414
Train Epoch: 1 [6400/60000] Loss: 0.081223
Train Epoch: 1 [12800/60000] Loss: 0.071415
Train Epoch: 1 [19200/60000] Loss: 0.064077
Train Epoch: 1 [25600/60000] Loss: 0.055545
Train Epoch: 1 [32000/60000] Loss: 0.049059
Train Epoch: 1 [38400/60000] Loss: 0.043199
Train Epoch: 1 [44800/60000] Loss: 0.042983
Train Epoch: 1 [51200/60000] Loss: 0.038389
Train Epoch: 1 [57600/60000] Loss: 0.039516
Test set: Average loss: 0.0000, Accuracy: 8657/10000 (86.57%)
Train Epoch: 2 [0/60000] Loss: 0.038534
Train Epoch: 2 [6400/60000] Loss: 0.040207
Train Epoch: 2 [12800/60000] Loss: 0.032802
Train Epoch: 2 [19200/60000] Loss: 0.030782
Train Epoch: 2 [25600/60000] Loss: 0.038660
Train Epoch: 2 [32000/60000] Loss: 0.026950
Train Epoch: 2 [38400/60000] Loss: 0.029296
Train Epoch: 2 [44800/60000] Loss: 0.022946
Train Epoch: 2 [51200/60000] Loss: 0.017221
Train Epoch: 2 [57600/60000] Loss: 0.038630
Test set: Average loss: 0.0000, Accuracy: 9053/10000 (90.53%)
Train 

##### Creating a UI to test the model

In [7]:
class MNISTApp:
    def __init__(self, master, model, device):
        self.master = master
        self.master.title("MNIST Digit Recognition")
        self.model = model
        self.device = device

        self.canvas = Canvas(master, width=200, height=200, bg='white')
        self.canvas.pack()

        self.predict_button = Button(master, text="Predict", command=self.predict_digit)
        self.predict_button.pack()

        self.clear_button = Button(master, text="Clear", command=self.clear_canvas)
        self.clear_button.pack()

        self.canvas.bind("<B1-Motion>", self.paint)

        self.image = PIL.Image.new("L", (28, 28), color="white")
        self.draw = ImageDraw.Draw(self.image)

    def paint(self, event):
        x1, y1 = (event.x - 4), (event.y - 4)
        x2, y2 = (event.x + 4), (event.y + 4)
        self.canvas.create_oval(x1, y1, x2, y2, fill='black', width=10)
        self.draw.ellipse([x1//7, y1//7, x2//7, y2//7], fill="black")

    def predict_digit(self):
        img = self.image.resize((28, 28))
        img = ImageOps.invert(img)
        img = transforms.ToTensor()(img).unsqueeze(0).to(self.device)

        self.model.eval()
        output = self.model(img)
        pred = output.argmax(dim=1)
        pred_digit = pred.item()

        result = f"Predicted Digit: {pred_digit}"
        print(result)
        self.master.title(result)

    def clear_canvas(self):
        self.canvas.delete("all")
        self.image = PIL.Image.new("L", (28, 28), color="white")
        self.draw = ImageDraw.Draw(self.image)

if __name__ == "__main__":
    root = Tk()
    app = MNISTApp(root, model, device)
    root.mainloop()


Predicted Digit: 1
Predicted Digit: 9
Predicted Digit: 2
Predicted Digit: 3
Predicted Digit: 8
Predicted Digit: 9
Predicted Digit: 7
Predicted Digit: 2
Predicted Digit: 9
Predicted Digit: 9
Predicted Digit: 0
Predicted Digit: 5
Predicted Digit: 0
Predicted Digit: 1
Predicted Digit: 1
Predicted Digit: 3
Predicted Digit: 3
Predicted Digit: 3
Predicted Digit: 3
Predicted Digit: 7
Predicted Digit: 7
Predicted Digit: 8
Predicted Digit: 3
Predicted Digit: 7
Predicted Digit: 5
Predicted Digit: 5
Predicted Digit: 6
Predicted Digit: 6
Predicted Digit: 6
Predicted Digit: 8
Predicted Digit: 3
Predicted Digit: 8
Predicted Digit: 8
Predicted Digit: 8
